In [2]:
#IMPORTAÇÃO DOS MODULOS E CHAMADA DAS TABELAS
import os
import pandas as pd
import glob
import pyodbc

In [ ]:
#FUNÇAO PARA CONEXAO COM O BANCO DE DADOS SQL
def executar_conexao():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Conexão ao BD realizada com Socesso!')
    return conexao
executar_conexao()

In [3]:
path = r'C:\Users\gleisson.santos\Downloads\LF - 26.02 a 25.03'
files_xls = os.listdir(path)

df = pd.DataFrame()
for f in files_xls:
    file_path = os.path.join(path, f)
    data = pd.read_html(file_path)
    data = data[1]
    data = data.dropna(axis=1, how='all')
    df = df.append(data)   
df



,SS,OS,Tipo,Especificação,Serviço,Unid Atual (OS),Equipe,Origem da SS,Localidade,Bairro,...,Encerramento,Mot de Encerr da OS,Sit da OS,Insumos,Func. Encerrou OS,SS Atendida,Tipo de Operação,Data/Hora Última Tramitação da OS,Ini do Serviço,Fim do Serviço
0,967060382,3,26 - MANUTENCAO DE HIDROMETRO,19 - REVISAO HIDROMETRO PARADO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,NaN,Interna,900 - SALVADOR UMB,18 - ITAPUA,...,28/02/2023 09:39:10,120 - HIDROMETRO NAO LOCALIZADO,Concluída Não Executada,0,33620 - PATRICIA DA SILVA SANTOS,Sim,Comercial,03/03/2022 14:28:18,NaN,NaN
1,974056340,2,26 - MANUTENCAO DE HIDROMETRO,2 - INSTALACAO DE HIDROMETRO,11 - INSTALACAO DE HIDROM.,88880068 - CONSORCIO NOVA BOLANDEIRA,228-A - IVAN HONORATO DOS SANTOS,Interna,900 - SALVADOR UMB,191 - BAIRRO DA PAZ,...,28/02/2023 11:27:35,8 - NAO PERMITIDO PELO USUARIO,Concluída Não Executada,0,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,09/09/2022 15:48:14,NaN,NaN
2,974846132,1,26 - MANUTENCAO DE HIDROMETRO,2 - INSTALACAO DE HIDROMETRO,11 - INSTALACAO DE HIDROM.,88880068 - CONSORCIO NOVA BOLANDEIRA,228-A - IVAN HONORATO DOS SANTOS,Interna,900 - SALVADOR UMB,191 - BAIRRO DA PAZ,...,28/02/2023 11:50:23,110 - RAMAL INTERNO,Concluída Não Executada,0,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,08/09/2022 16:53:36,NaN,NaN
3,974846138,1,26 - MANUTENCAO DE HIDROMETRO,2 - INSTALACAO DE HIDROMETRO,11 - INSTALACAO DE HIDROM.,88880068 - CONSORCIO NOVA BOLANDEIRA,228-A - IVAN HONORATO DOS SANTOS,Interna,900 - SALVADOR UMB,191 - BAIRRO DA PAZ,...,28/02/2023 11:25:59,121 - RAMAL PROFUNDO NAO LOCALIZADO,Concluída Não Executada,0,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,08/09/2022 16:53:36,NaN,NaN
4,975370954,1,26 - MANUTENCAO DE HIDROMETRO,2 - INSTALACAO DE HIDROMETRO,11 - INSTALACAO DE HIDROM.,88880068 - CONSORCIO NOVA BOLANDEIRA,228-A - IVAN HONORATO DOS SANTOS,Interna,900 - SALVADOR UMB,191 - BAIRRO DA PAZ,...,28/02/2023 11:32:37,8 - NAO PERMITIDO PELO USUARIO,Concluída Não Executada,0,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,21/09/2022 08:45:53,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,981263842,1,45 - VERIFICACAO DE FALTA DE AGUA,2 - FALTA DE AGUA NA RUA / LOGRADOURO,365 - VERIF FALTA AGUA RUA,88880068 - CONSORCIO NOVA BOLANDEIRA,NaN,Cliente,700 - LAURO DE FREITAS,3 - CAJI,...,28/02/2023 16:14:11,18 - SERVICO EXECUTADO P/ OUTRA OS,Concluída Não Executada,0,696849 - RONLISON CONCEICAO DA COSTA,Não,Água,28/02/2023 16:04:19,NaN,NaN
119,981264232,1,25 - MANUTENCAO DE ESGOTO,2 - EXTRAVASAMENTO DE ESGOTO RAMAL - CI,438 - DES RED ESG E PVS C/EQUIP,88880068 - CONSORCIO NOVA BOLANDEIRA,221-E - EDMILSON GOLVEIA ALMEIDA,Cliente,700 - LAURO DE FREITAS,6 - VILA PRAIANA,...,28/02/2023 17:29:02,9 - NORMALIZADO,Concluída Não Executada,0,204589 - GLEISSON DE JESUS SANTOS,Não,Esgoto,28/02/2023 15:34:43,NaN,NaN
120,981264721,1,25 - MANUTENCAO DE ESGOTO,4 - EXTRAVASAMENTO - RETORNO IMOVEL,440 - DES RAM PREDI ESG C/EQUIP,88880068 - CONSORCIO NOVA BOLANDEIRA,221-E - EDMILSON GOLVEIA ALMEIDA,Cliente,700 - LAURO DE FREITAS,13 - ITINGA,...,28/02/2023 16:50:50,9 - NORMALIZADO,Concluída Não Executada,0,204589 - GLEISSON DE JESUS SANTOS,Não,Esgoto,28/02/2023 15:38:16,NaN,NaN
121,981270282,1,43 - VAZAMENTO DE AGUA,3 - VAZAMENTO NO HIDROMETRO,290 - VAZAMENTO NO HIDROMETRO,88880068 - CONSORCIO NOVA BOLANDEIRA,132A - MARCOS CASSIO DE JESUS SILVA,Cliente,700 - LAURO DE FREITAS,13 - ITINGA,...,28/02/2023 17:28:50,9 - NORMALIZADO,Concluída Não Executada,0,696849 - RONLISON CONCEICAO DA COSTA,Não,Água,28/02/2023 16:53:05,NaN,NaN


In [ ]:
df.to_excel('LF - 26.02 a 25.03.xlsx', index=False)

In [ ]:
#INSERT DA TABALA TRATADA AO BANCO DE DADOS SQL
cursor = executar_conexao().cursor()
for index, row in df.iterrows():
    cursor.execute("INSERT INTO Material (SS, OS, Serviço, Equipe, Localidade, Bairro, Logradouro, Encerramento, ObsOS, ObsEnc) values(?,?,?,?,?,?,?,?,?,?)",
                row.SS, row.OS, row.Serviço, row.Equipe, row.Localidade,  row.Bairro, row.Logradouro, row.Encerramento, row.ObsOS, row.ObsEnc)

cursor.commit()
cursor.close()

In [ ]:
#APAGAR REGISTROS NA TABELA
cursor = executar_conexao().cursor()
cursor.execute('TRUNCATE TABLE Material')
cursor.commit()

In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
import pandas as pd
import os
import concurrent.futures


import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
import pandas as pd
import os
import concurrent.futures

def select_folder():
    global folder_path
    folder_selected = filedialog.askdirectory()
    folder_path.set(folder_selected)

def process_files():
    global folder_path, progress_bar, df

    path = folder_path.get()

    if path:
        files_xls = os.listdir(path)

        dfs = []
        # Definir função que processa um arquivo
        def process_file(f):
            file_path = os.path.join(path, f)
            data = pd.read_html(file_path)
            data = data[1]
            data = data.dropna(axis=1, how='all')
            return data

        # Definir função que atualiza a barra de progresso
        def update_progress(progress):
            progress_bar['value'] = progress
            root.update()

        # Processar arquivos em paralelo usando a biblioteca concurrent.futures
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []
            for f in files_xls:
                future = executor.submit(process_file, f)
                futures.append(future)

            # Atualizar barra de progresso enquanto os arquivos são processados
            total_files = len(files_xls)
            files_processed = 0
            while files_processed < total_files:
                progress = int(100 * files_processed / total_files)
                update_progress(progress)
                files_processed = sum([future.done() for future in futures])
            
            # Combinar resultados em um único DataFrame
            df = pd.concat([future.result() for future in futures])

        # Exibir mensagem de conclusão
        messagebox.showinfo("Processamento Concluído", f"Processamento de arquivos concluído. Foram processados {len(files_xls)} arquivos.")

    else:
        # Exibir mensagem de erro
        messagebox.showerror("Erro", "Selecione uma pasta para processar.")


def save_file():
    global df
    
    # Abrir janela de seleção de arquivo
    file_path = filedialog.asksaveasfilename(defaultextension='.xlsx')
    
    # Verificar se um arquivo foi selecionado
    if file_path:
        # Salvar arquivo Excel
        df.to_excel(file_path, index=False)
        
        # Exibir mensagem de conclusão
        messagebox.showinfo("Arquivo Salvo", f"O arquivo {file_path} foi salvo com sucesso!")
        
    else:
        # Exibir mensagem de erro
        messagebox.showerror("Erro", "Selecione um arquivo para salvar.")

# criar a interface gráfica
root = tk.Tk()
root.geometry("500x300")
root.configure(bg="#fff")
root.title("Junção de arquivos")


# adicionar imagem da empresa
img = tk.PhotoImage(file="imagem_empresa.gif")
tk.Label(root, image=img, bg="#FFF").place(x=10, y=10)

# Adicionar entrada de texto para a pasta
folder_path = tk.StringVar()
tk.Label(root, text="Selecione uma pasta:", bg="#fff").place(x=170, y=30)
tk.Entry(root, textvariable=folder_path, width=50).place(x=175, y=20, anchor='w')

# Adicionar botão para selecionar a pasta
tk.Button(root, text="Selecionar pasta", command=select_folder).place(x=270, y=55)

# Adicionar botão para processar os arquivos
tk.Button(root, text="Processar arquivos", command=process_files).place(x=310, y=95)

tk.Button(root, text="Salvar arquivo   ", command=save_file).place(x=380, y=55)


# Adicionar barra de progresso
progress_bar = ttk.Progressbar(root, mode='determinate', maximum=100, value=0)
progress_bar.place(width=300, relx=0.5, rely=0.9, anchor='center')


# Loop principal
root.mainloop()
